In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

path = 'inputs/'

In [2]:
def lag_feature(df, feat, lags=[1], suf='_lag', drop=True):
    for lag in lags:
        df[f'{feat}{suf}{lag}'] = df[feat].shift(lag)
    if drop:
        df = df.drop(columns=[feat])
    return df



In [3]:
df_store = pd.read_csv(path + 'stores.csv',
                         usecols=['store_nbr', 'city', 'state', 'type', 'cluster'],
                         dtype={'store_nbr': 'category', 'city': 'category', 'state': 'category', 'cluster': 'category'})

df_trans = pd.read_csv(path + 'transactions.csv',
                       usecols=['store_nbr', 'date', 'transactions'],
                       dtype={'store_nbr': 'category', 'transactions': 'float32'},
                       parse_dates=['date'],
                       date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                       infer_datetime_format=True)

df_holiday = pd.read_csv(path + 'holidays_events.csv',
                         index_col=['date'],
                         usecols=['date', 'type', 'locale', 'locale_name', 'transferred'],
                         dtype={'type' : 'str', 'locale_name': 'str', 'locale': 'str', 'transferred': 'boolean'},
                         parse_dates=['date'],
                         date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                         infer_datetime_format=True)

df_oil = pd.read_csv(path + 'oil.csv',
                     index_col=['date'],
                     parse_dates=['date'],
                     date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                     infer_datetime_format=True)

df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       index_col=['date', 'store_nbr', 'family'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32', 'onpromotion': 'uint32'},
                       parse_dates=['date'],
                       date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                       infer_datetime_format=True)

train_easy = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32', 'onpromotion': 'uint32'},
                       parse_dates=['date'],
                       date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                       infer_datetime_format=True)

df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date', 'onpromotion'],
                      index_col=['date', 'store_nbr', 'family'],
                      dtype={'store_nbr': 'category', 'family': 'category', 'onpromotion': 'uint32'},
                      parse_dates=['date'],
                      date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                      infer_datetime_format=True)

df_data = {
    'trsimp': train_easy,
    'holiday_load': df_holiday,
    'oil_load': df_oil,
    'train_load': df_train,
    'test_load': df_test,
    'store_load': df_store,
    'trans_load': df_trans,
}

In [4]:
stores = df_data['store_load'].copy()
uniq = stores.city.value_counts()[stores.city.value_counts() == 1].index.tolist()
stores['uniq_store'] = stores.city.apply(lambda x: 1 if x in uniq else 0)
###WHY
new_stores = [19, 20, 21, 28, 35, 41, 51, 52]
stores['new_store'] = stores['store_nbr'].apply(lambda x: 1 if x in new_stores else 0)
stores = stores.rename(columns={'type': 'store'})
stores = stores.set_index('store_nbr')
df_data['str_mod'] = stores

In [5]:
oil = df_data['oil_load'].copy()
oil = oil.resample('D').mean().interpolate(limit_direction='backward').reset_index()
oil = lag_feature(oil, 'dcoilwtico', [1, 2, 3, 4], drop=False)
oil['wk_avg_oil'] = oil['dcoilwtico'].rolling(7).mean()
oil['biwk_avg_oil'] = oil['dcoilwtico'].rolling(14).mean()
oil['mnth_avg_oil'] = oil['dcoilwtico'].rolling(30).mean()
oil['bimnth_avg_oil'] = oil['dcoilwtico'].rolling(60).mean()
oil.dropna(inplace=True)
oil = oil.set_index('date')
df_data['oil_mod'] = oil

In [6]:
date_range = pd.Series(pd.date_range('2013-01-01', '2017-08-31').to_period('D'), name='date')
cal = pd.DataFrame(index=date_range)
cal = cal.merge(df_data['holiday_load'], how='left', on=['date'])
cal.loc[cal.isna().any(axis=1),
           ['type', 'locale', 'locale_name', 'transferred']] = 'Work Day', 'National', 'Ecuador', False
cal['wd'] = True
cal.loc[(cal.type == 'Bridge') | (cal.type == 'Transfer'), 'wd'] = False

cal.loc[(((cal.type == 'Additional')|(cal.type == 'Holiday')) & (cal.transferred == False)), 'wd'] = False
easter = ['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']
easter = [pd.to_datetime(date, format='%Y-%m-%d').to_period('D') for date in easter]
cal['isevnt'] = False
cal.loc[cal.index.get_level_values('date').isin(easter), 'isevnt'] = True
cal = pd.get_dummies(cal, columns=['type'])
cal = pd.get_dummies(cal, columns=['locale'])
cal.drop(columns=['locale_name', 'transferred'], inplace=True)
agg=lambda x: np.bitwise_or.reduce(x.values)
cal = cal.groupby('date').agg(agg)
df_data['cal'] = cal

In [7]:
df_data.keys()

dict_keys(['trsimp', 'holiday_load', 'oil_load', 'train_load', 'test_load', 'store_load', 'trans_load', 'str_mod', 'oil_mod', 'cal'])

In [8]:
train_df = df_data['train_load'].copy()
test_df = df_data['test_load'].copy()
test_df.sales = np.nan

df = pd.concat([train_df, test_df], axis=0)
df = df.merge(df_data['str_mod'], how='left', left_index=True, right_index=True)
df.drop(df.filter(regex='_y$').columns, axis=1, inplace=True)
df['closed'] = df.groupby(by=['date', 'store_nbr'])['sales'].transform(lambda x:1 if x.sum() ==0 else 0)
# stores in test set aren't close, just have 0 sales
df.loc[
    ((df.index.get_level_values('date').year==2017) &
     (df.index.get_level_values('date').month==8) & (
             df.index.get_level_values('date').day>=16)) , 'closed'] = df['closed'].apply(lambda x: 0)

## Holidays
df = df.merge(df_data['cal'], how='left', left_index=True, right_index=True, suffixes=('', '_y'))
df.drop(df.filter(regex='_y$').columns, axis=1, inplace=True)

In [9]:
## Oil
df = df.merge(df_data['oil_mod'], how='left', left_index=True, right_index=True)

In [10]:
df['year'] = df.index.get_level_values('date').year.astype('int')
df['quarter'] = df.index.get_level_values('date').quarter.astype('int')
df['day'] = df.index.get_level_values('date').day.astype('int')

df['weekofyear'] = df.index.get_level_values('date').week.astype('int')
df['dayofweek'] = df.index.get_level_values('date').day_of_week.astype('int')
df['isweekend'] = df.dayofweek.apply(lambda x: 1 if x in (5,6) else 0)

df['newyear'] = df.index.get_level_values('date').dayofyear == 1
df['startschool'] = df.index.get_level_values('date').month.isin((4,5,8,9))
df['weekend'] = df.index.get_level_values('date').dayofweek.isin((5,6))

In [11]:
df['payday'] = ((df.index.get_level_values('date').to_timestamp().is_month_end) | (df.index.get_level_values('date').day == 15))

quake_dates = [d.to_period('D') for d in pd.date_range('2016-04-16', periods=20, freq='D').tolist()]
df['quake'] = df.index.get_level_values('date').isin(quake_dates)

In [12]:
df = pd.get_dummies(df, columns=['year'], drop_first=True)
df = pd.get_dummies(df, columns=['quarter'], drop_first=True)
df = pd.get_dummies(df, columns=['dayofweek'], drop_first=True)

df['wk_avg_promo'] = df['onpromotion'].rolling(7).mean()
df['biwk_avh_promo'] = df['onpromotion'].rolling(14).mean()
df['mnth_avg_promo'] = df['onpromotion'].rolling(30).mean()
df['bimnth_avg_promo'] = df['onpromotion'].rolling(60).mean()

df.drop(columns=['city', 'cluster', 'state', 'store'], inplace=True)

In [13]:
from statsmodels.tsa.deterministic import DeterministicProcess
from statsmodels.tsa.deterministic import CalendarFourier

fourier_annual = CalendarFourier(freq='A', order=5)
fourier_monthly = CalendarFourier(freq='M', order=2)
fourier_weekly = CalendarFourier(freq='W', order=3)

terms = [fourier_annual, fourier_monthly, fourier_weekly]

dp = DeterministicProcess(index=df.index.get_level_values('date'),
                          order=1,
                          seasonal=True,
                          constant=False,
                          additional_terms=terms,
                          drop=True)

df_dp = dp.in_sample()
df = pd.concat([df.reset_index(level=['store_nbr', 'family']), df_dp], axis=1)

df = df.set_index(['store_nbr', 'family'], append=True)

In [14]:
import time
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_log_error
from IPython.core import display as ICD
from upgini import FeaturesEnricher, SearchKey, ModelTaskType
from upgini.metadata import CVType

def _scorer(y_true, y_predicted):
    y_true = np.exp(y_true)-1
    y_predicted = np.exp(y_predicted)-1
    return mean_squared_log_error(y_true, y_predicted, squared=False)

scorer = make_scorer(_scorer, greater_is_better=False)

def index_timeseries_train_test_split(df, date_index=False, date_format='%Y-%m-%d', earliest_train_date=None, split_date=None, last_test_date=None, verbose=True):
    dates = df.index.get_level_values('date').to_timestamp()

    if earliest_train_date:
        earliest_train_datetime = datetime.strptime(earliest_train_date, date_format)
        df = df.loc[[earliest_train_datetime<=date for date in dates]]
        dates = df.index.get_level_values('date').to_timestamp()

    if last_test_date:
        last_test_datetime = datetime.strptime(last_test_date, date_format)
        df = df.loc[[date<last_test_datetime for date in dates]]
        dates = df.index.get_level_values('date').to_timestamp()

    split_datetime = datetime.strptime(split_date, date_format)
    train_dates = [date<split_datetime for date in dates]
    test_dates = np.invert(train_dates)

    train_df = df.loc[train_dates]
    test_df =  df.loc[test_dates]

    if verbose:
        print('================================')
        print('Training Dataset last rows')
        ICD.display(train_df.tail(3))
        print('================================')
        print('Test Dataset first rows')
        ICD.display(test_df.head(3))

    return train_df, test_df


def separate_X_and_y(df, target='sales'):
    y = df[target].values
    X = df.drop(columns=target)
    return X, y


def unstack_transformation(X, y, group_by='date', target='sales', levels=['store_nbr', 'family']):
    df = X.copy()
    df[target] = y
    y = df.loc[:, target].unstack(levels)
    X = df.drop(columns=[target]).groupby(by=group_by).first()
    return X, y


def create_sample_weights(X, target_date, weight=0.9):
    extra_weight_days = X.index.get_level_values('date') > target_date
    return np.array(list(map(lambda x: np.exp(-weight) if x == 0 else 1, extra_weight_days.astype('int'))))


def get_closed_stores(df, last_days=15):
    closed_stores_df = df.groupby(['family']).tail(last_days).groupby(['family'])['sales'].sum().reset_index()
    closed_stores_df = closed_stores_df[closed_stores_df['sales'] == 0].drop('sales', axis = 1)
    closed_stores_df = closed_stores_df[closed_stores_df['family'] != "SCHOOL AND OFFICE SUPPLIES"]
    closed_stores_df = closed_stores_df.set_index(['family'])
    closed_stores_df['closed'] = True
    return closed_stores_df


def apply_zero_forecasting(df, closed_stores_df):
    df = df.merge(closed_stores_df, how='left',  left_index=True, right_index=True)
    df.loc[df['closed']==True, 'sales']=0
    df.drop(columns=['closed'], inplace=True)
    return df


def timeit(function):
    def wrapper(*args, **kwargs):
        ts = time.perf_counter()
        func = function(*args, **kwargs)
        te = time.perf_counter ()
        run_time = time.perf_counter() - ts
        print(f'Run Time: {run_time} secs')
        return func
    return wrapper


def plot_feature_importance(feature_importances, feature_names):
    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(feature_names, feature_importances):
        feats[feature] = importance #add the name/value pair

    importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
    importances.sort_values(by='Gini-importance').plot(kind='bar').show()



def plot_feature_elimination_score(min_features_to_select, grid_score):
    plt.figure()
    plt.xlabel('Number of features selected')
    plt.ylabel('Cross validation score')
    plt.plot(
        range(min_features_to_select, len(grid_score) + min_features_to_select),
        grid_score
    )
    plt.show()


def plot_predition_timeseries(x, y_actual, y_pred):
    df = pd.DataFrame()
    df['date'], df['actual'], df['prediction'] = x, y_actual, y_pred
    df = df.groupby(['date'], as_index=False).sum().reset_index()
    df['date'] = df['date'].apply(lambda row: row.to_timestamp())
    title = 'Actual vs Prediction'
    df.plot.line(x='date', y=['actual', 'prediction'], title=title).show()


def transform_prediction_to_df(X, y, prediction):
    return pd.DataFrame(pd.DataFrame(prediction,
                        index=X.index,
                        columns=y.columns).stack(['store_nbr'])).rename(columns={0: 'sales'})


def per_family_df_generator(train_df, test_df, verbose=False):
    families = sorted(train_df.index.get_level_values('family').unique())
    for _, family in zip(tqdm(range(len(families))), families):
        if verbose:
            print('='*40)
            print(f'Family: {family}')
        # Train DF
        train_family_df = train_df[train_df.index.get_level_values('family')==family]
        train_family_df.index = train_family_df.index.droplevel('family')
        # Test DF
        test_family_df = test_df[test_df.index.get_level_values('family')==family]
        test_family_df.index = test_family_df.index.droplevel('family')
        yield family, train_family_df, test_family_df

In [15]:
class UpginiEnricher:
    def __init__(self):
        self.enricher = FeaturesEnricher(
        search_keys={
            "date": SearchKey.DATE
        },
        country_code = "EC",
        model_task_type = ModelTaskType.REGRESSION,
        cv = CVType.time_series,
        )

    def fit(self, df):
        df = df.reset_index()[['date','sales']].groupby('date', as_index=False).mean()
        y = df['sales'].values
        X = df.drop(columns=['sales'])

        self.enricher.fit(
            X,
            y,
            calculate_metrics=True,
            max_features=3)
        return self

    def transform(self, df):
        return self.enricher.transform(df.copy().reset_index(),
                                       keep_input=True,
                                       max_features=4,
                                    ).set_index(['date', 'store_nbr'])

In [16]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression


class MultiOutBaggingExtraTreesRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,
                extra_trees_n_estimators=200,
                bagging_n_estimators=10,
                log_target_transform=True,
                multioutput=True,
                n_jobs=-1,
                random_state=42):
        super().__init__()
        _r1 = ExtraTreesRegressor(n_estimators=extra_trees_n_estimators,
                                  n_jobs=n_jobs,
                                  random_state=random_state)
        _estimator = BaggingRegressor(base_estimator=_r1,
                                      n_estimators=bagging_n_estimators,
                                      bootstrap=True,
                                      n_jobs=n_jobs,
                                      random_state=random_state)
        if multioutput:
            _estimator = MultiOutputRegressor(_estimator)
        self.estimator_ = _estimator
        self.log_target_transform = log_target_transform
        self.feature_importances_ = None
        self.bagging_n_estimators = bagging_n_estimators
        self.extra_trees_n_estimators = extra_trees_n_estimators
        self.multioutput = multioutput
        self.n_jobs = n_jobs
        self.random_state = random_state

    def fit(self, X, y, sample_weight=None):
        if self.log_target_transform:
            y = np.log1p(y)
        self.estimator_.fit(X, y, sample_weight)

        if self.multioutput:
            feature_importances_ = []
            for n_output_estimators in self.estimator_.estimators_:
                feature_importances_.append(np.mean([
                tree.feature_importances_ for tree in n_output_estimators.estimators_], axis=0))
            self.feature_importances_ = np.mean(feature_importances_, axis=0)
        else:
            self.feature_importances_ = np.mean([
                tree.feature_importances_ for tree in self.estimator_.estimators_], axis=0)

        return self

    def predict(self, X, y=None):
        prediction = self.estimator_.predict(X)
        if self.log_target_transform:
            prediction = (np.exp(prediction)-1)
        return prediction.clip(0)

    def score(self, X, y):
        prediction = self.predict(X, y)
        return mean_squared_log_error(y, prediction, squared=False)

In [17]:
from sklearn.metrics import mean_squared_log_error

def family_prediction_generator(train_df, test_df, estimator_params={}, verbose=True):
    for family, train_family_df, test_family_df in per_family_df_generator(train_df, test_df, verbose=verbose):

        if family in ['LIQUOR,WINE,BEER', 'CLEANING']:
            enricher = UpginiEnricher()
            enricher.fit(train_family_df)
            train_family_df = enricher.transform(train_family_df)
            test_family_df = enricher.transform(test_family_df)

        # =====================================
        # Create multi-output (with unstack)
        # =====================================
        X_train, y_train = separate_X_and_y(train_family_df)
        X_train, y_train = unstack_transformation(X_train, y_train, levels=['store_nbr'])
        X_test, y_test = separate_X_and_y(test_family_df)
        X_test, y_test = unstack_transformation(X_test, y_test, levels=['store_nbr'])

        # =====================================
        # Train
        # =====================================
        no_multiple_outputs = ['BABY CARE', 'GROCERY II','HOME AND KITCHEN I','HOME AND KITCHEN II', 'SCHOOL AND OFFICE SUPPLIES']
        if family in no_multiple_outputs:
            estimator = MultiOutBaggingExtraTreesRegressor(**estimator_params, multioutput=False)
        else:
            estimator = MultiOutBaggingExtraTreesRegressor(**estimator_params)

        weights = create_sample_weights(X_train, '2017-07-01')
        estimator.fit(X_train, y_train, sample_weight=weights)

        feature_importances = estimator.feature_importances_
        feature_names = X_train.columns
        #plot_feature_importance(feature_importances, feature_names)

        # =====================================
        # Predictions (for Training & Test)
        # =====================================
        train_pred = estimator.predict(X_train)
        train_pred_df = transform_prediction_to_df(X_train, y_train, train_pred)
        train_pred_df['family'] = family
        train_pred_df = train_pred_df.set_index('family', append=True)

        test_pred = estimator.predict(X_test)
        test_pred_df = transform_prediction_to_df(X_test, y_test, test_pred)
        test_pred_df['family'] = family
        test_pred_df = test_pred_df.set_index('family', append=True)

        # =====================================
        # Scoring
        # =====================================
        family_train_score = mean_squared_log_error(train_family_df['sales'].values,
                                                    train_pred_df['sales'].values,
                                                    squared=False)
        if verbose:
            print(f'RMSLE || TRAINING Score: {family_train_score}')

        if not any(np.isnan(test_family_df['sales'].values)):
            # Print score only in evaluation
            family_test_score = mean_squared_log_error(test_family_df['sales'].values,
                                                       test_pred_df['sales'].values,
                                                       squared=False)
            if verbose:
                print(f'RMSLE || TEST Score: {family_test_score}')
        yield train_pred_df, test_pred_df

In [18]:
train_df, test_df = index_timeseries_train_test_split(df=df,
                                                      date_index='date',
                                                      date_format='%Y-%m-%d',
                                                      earliest_train_date='2013-03-01',
                                                      split_date='2017-07-15', # '2016-08-01',
                                                      last_test_date='2017-08-01',  #'2016-09-15',
                                                      verbose=False)

In [20]:
%%time
# NOTE: Only for notebook run in Kaggle I decreased the numbers in model in order
# to have a quicker run.
estimator_params = {
                    'extra_trees_n_estimators': 15,
                    'bagging_n_estimators': 15,
    }

training_total_pred_df = pd.DataFrame()
test_total_pred_df = pd.DataFrame()
for train_pred_df, test_pred_df in family_prediction_generator(train_df, test_df, estimator_params):
    # Keep prediction for later plotting
    training_total_pred_df = pd.concat([training_total_pred_df, train_pred_df])
    test_total_pred_df = pd.concat([test_total_pred_df, test_pred_df])

  0%|          | 0/33 [00:00<?, ?it/s]

Family: AUTOMOTIVE
RMSLE || TRAINING Score: 0.2402763045347676
RMSLE || TEST Score: 0.5088356014593546
Family: BABY CARE
RMSLE || TRAINING Score: 0.08458465450116415
RMSLE || TEST Score: 0.2983051692571459
Family: BEAUTY
RMSLE || TRAINING Score: 0.21387906404430582
RMSLE || TEST Score: 0.5038332257124054
Family: BEVERAGES
RMSLE || TRAINING Score: 0.1280445070103752
RMSLE || TEST Score: 0.20474457696268664
Family: BOOKS
RMSLE || TRAINING Score: 0.05765217833369131
RMSLE || TEST Score: 0.14214367700742941
Family: BREAD/BAKERY
RMSLE || TRAINING Score: 0.10542216411618979
RMSLE || TEST Score: 0.1649985593328745
Family: CELEBRATION
RMSLE || TRAINING Score: 0.19812098645683798
RMSLE || TEST Score: 0.5749634288597429
Family: CLEANING


Column name,Status,Description
country_iso_code,All valid,All values in this column are good to go
date,All valid,All values in this column are good to go
target,All valid,All values in this column are good to go


Running search request with search_id=085302cd-5ba1-431b-a8b7-6cd4445dd322
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


8 relevant feature(s) found with the search keys: ['date', 'country_iso_code']


/Users/jarret/opt/anaconda3/envs/CSCI-567-FinalProject/lib/python3.10/site-packages/upgini/features_enricher.py:1476: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  display(self.features_info.head(60).style.hide_index())


...

,match_rate,enriched mean_squared_error
,,
train,100.0,1811.071926


Column name,Status,Description
country_iso_code,All valid,All values in this column are good to go
date,All valid,All values in this column are good to go


Running search request with search_id=3b6d58e3-1cf7-4206-8431-e31be20ece58
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done


Column name,Status,Description
country_iso_code,All valid,All values in this column are good to go
date,All valid,All values in this column are good to go


Running search request with search_id=636e2a46-3a32-4651-94de-6bef0250be6e
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done
RMSLE || TRAINING Score: 0.3240668325798615
RMSLE || TEST Score: 0.5332175286732441
Family: MAGAZINES
RMSLE || TRAINING Score: 0.17572440740535689
RMSLE || TEST Score: 0.5103513187971324
Family: MEATS
RMSLE || TRAINING Score: 0.11748987983201718
RMSLE || TEST Score: 0.18650998648101225
Family: PERSONAL CARE
RMSLE || TRAINING Score: 0.12077687078948304
RMSLE || TEST Score: 0.22819562637992907
Family: PET SUPPLIES
RMSLE || TRAINING Score: 0.16640449730093948
RMSLE || TEST Score: 0.5000028967039544
Family: PLAYERS AND ELECTRONICS
RMSLE || TRAINING Score: 0.1789956993370156
RMSLE || TEST Score: 0.49256560865285104
Family: POULTRY
RMSLE || TRAINING Score: 0.12630387033787846
RMSLE || TEST Score: 0.20846334372663525
Family: PREPARED FOODS
RMSLE || TRAINING Score: 0.13572571124494462
RMSLE || TEST Score:

In [22]:
print('===> Print All stores Training Prediction')
training_total_pred_df = training_total_pred_df.sort_values(['date', 'store_nbr', 'family'])
training_score = mean_squared_log_error(train_df['sales'].values, training_total_pred_df['sales'].values, squared=False)
# plot_predition_timeseries(x=training_total_pred_df.index.get_level_values('date'),
#                             y_actual=train_df['sales'].values,
#                             y_pred=training_total_pred_df['sales'].values)

print('===> Print All stores Test Prediction')
test_total_pred_df = test_total_pred_df.sort_values(['date', 'store_nbr', 'family'])
test_score = mean_squared_log_error(test_df['sales'].values, test_total_pred_df['sales'].values, squared=False)
# plot_predition_timeseries(x=test_total_pred_df.index.get_level_values('date'),
#                             y_actual=test_df['sales'].values,
#                             y_pred=test_total_pred_df['sales'].values)

print(f'RMSLE || TRAINING Score: {training_score} | TEST Score: {test_score}')

===> Print All stores Training Prediction
===> Print All stores Test Prediction
RMSLE || TRAINING Score: 0.1778913458624602 | TEST Score: 0.43016112367140596


In [ ]:
%%time
train_df, test_df = index_timeseries_train_test_split(df=df,
                                                      date_index='date',
                                                      date_format='%Y-%m-%d',
                                                      earliest_train_date='2013-03-01',
                                                      split_date='2017-08-16',
                                                      verbose=False)

# Get stopped families for each store
closed_stores_df = get_closed_stores(train_df, last_days=12)


training_total_pred_df = pd.DataFrame()
test_total_pred_df = pd.DataFrame()
for train_pred_df, test_pred_df in family_prediction_generator(train_df, test_df):
    # Keep prediction for submission
    training_total_pred_df = pd.concat([training_total_pred_df, train_pred_df])
    test_total_pred_df = pd.concat([test_total_pred_df, test_pred_df])

Training Dataset last rows


/var/folders/qs/7tn9p_8910q1ksx2jdch1wmr0000gn/T/ipykernel_88384/351393295.py:41: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  ICD.display(train_df.tail(3))


sales  onpromotion  \
date       store_nbr family                                                 
2017-08-15 9         PRODUCE                     2419.729004          148   
                     SCHOOL AND OFFICE SUPPLIES   121.000000            8   
                     SEAFOOD                       16.000000            0   

                                                 uniq_store  new_store  \
date       store_nbr family                                              
2017-08-15 9         PRODUCE                              0          0   
                     SCHOOL AND OFFICE SUPPLIES           0          0   
                     SEAFOOD                              0          0   

                                                 closed     wd  isevnt  \
date       store_nbr family                                              
2017-08-15 9         PRODUCE                          0  False   False   
                     SCHOOL AND OFFICE SUPPLIES       0  False   False   
                     SEAFOOD                          0  False   False   

                                                 type_Additional  type_Bridge  \
date       store_nbr family                                                     
2017-08-15 9         PRODUCE                                   0            0   
                     SCHOOL AND OFFICE SUPPLIES                0            0   
                     SEAFOOD                                   0            0   

                                                 type_Event  ...  \
date       store_nbr family                                  ...   
2017-08-15 9         PRODUCE                              0  ...   
                     SCHOOL AND OFFICE SUPPLIES           0  ...   
                     SEAFOOD                              0  ...   

                                                 sin(1,freq=M)  cos(1,freq=M)  \
date       store_nbr family                                                     
2017-08-15 9         PRODUCE                          0.299363      -0.954139   
                     SCHOOL AND OFFICE SUPPLIES       0.299363      -0.954139   
                     SEAFOOD                          0.299363      -0.954139   

                                                 sin(2,freq=M)  cos(2,freq=M)  \
date       store_nbr family                                                     
2017-08-15 9         PRODUCE                         -0.571268       0.820763   
                     SCHOOL AND OFFICE SUPPLIES      -0.571268       0.820763   
                     SEAFOOD                         -0.571268       0.820763   

                                                 sin(1,freq=W-SUN)  \
date       store_nbr family                                          
2017-08-15 9         PRODUCE                              0.781831   
                     SCHOOL AND OFFICE SUPPLIES           0.781831   
                     SEAFOOD                              0.781831   

                                                 cos(1,freq=W-SUN)  \
date       store_nbr family                                          
2017-08-15 9         PRODUCE                               0.62349   
                     SCHOOL AND OFFICE SUPPLIES            0.62349   
                     SEAFOOD                               0.62349   

                                                 sin(2,freq=W-SUN)  \
date       store_nbr family                                          
2017-08-15 9         PRODUCE                              0.974928   
                     SCHOOL AND OFFICE SUPPLIES           0.974928   
                     SEAFOOD                              0.974928   

                                                 cos(2,freq=W-SUN)  \
date       store_nbr family                                          
2017-08-15 9         PRODUCE                             -0.222521   
                     SCHOOL AND OFFICE SUPPLIES          -0.222521   
                     SEAFOOD       

Test Dataset first rows


/var/folders/qs/7tn9p_8910q1ksx2jdch1wmr0000gn/T/ipykernel_88384/351393295.py:44: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  ICD.display(test_df.head(3))


...

RMSLE || TRAINING Score: 0.1957117985797875
Family: LINGERIE


/Users/jarret/opt/anaconda3/envs/CSCI-567-FinalProject/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/jarret/opt/anaconda3/envs/CSCI-567-FinalProject/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/jarret/opt/anaconda3/envs/CSCI-567-FinalProject/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/jarret/opt/anaconda3/envs/CSCI-567-FinalProject/lib/python3.10/site-packages/joblib/externals/loky/process_executor.p

RMSLE || TRAINING Score: 0.23976024792481768
Family: LIQUOR,WINE,BEER


Column name,Status,Description
country_iso_code,All valid,All values in this column are good to go
date,All valid,All values in this column are good to go
target,All valid,All values in this column are good to go


Running search request with search_id=b795032f-abb5-402f-85fa-5742d821e864
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


In [ ]:
test_total_pred_df = test_total_pred_df.sort_values(['date', 'store_nbr', 'family'])

# Zero the sales for stopped families
test_total_pred_df= apply_zero_forecasting(test_total_pred_df, closed_stores_df)

ids = pd.read_csv(path + 'test.csv')['id']
test_total_pred_df['id'] = ids.values

submit_df = test_total_pred_df[['id', 'sales']]

if len(submit_df) != 28512: # Minor check before submission :-)
    raise Exception(f'The rows ({len(submit_df)}) of the export are not correct')
else:
    current_dateTime = datetime.now()
    latest_filename = 'submission.csv'
    submit_df.to_csv(latest_filename, index=True)
